1. Вытягивать словосочетания и выводить в предсказание следующее за ним слово / сет слов.
3. Правильно разбивать на предложения
2. Применить модели, выработать стратегии для ответов на различные типы вопросов.
2. Паттерны + нейросеть

In [83]:
import pandas as pd
import tqdm
import re, os

import string, nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np

from sklearn.metrics import f1_score

In [84]:
data = pd.read_csv('../data/train_task_b.csv')
data

,paragraph_id,question_id,paragraph,question,answer
0,14754,60544,Первые упоминания о строении человеческого тел...,Где встречаются первые упоминания о строении ч...,в Древнем Египте
1,13859,1604,Телескоп имеет модульную структуру и содержит ...,Как называется корректирующая оптическая систе...,COSTAR
2,8310,30350,Критики теории Вегенера поставили во главу угл...,Какая теория была отвергнута после смерти Веге...,теория дрейфа материков
3,8766,53270,При нагревании кусочки янтаря становятся очень...,Чему не уступают по красоте изделия из прессов...,изделиям из монолитных камней
4,14719,75698,Нисходящие дифтонги со слабым гласным /i/ и /u...,На какие группы классифицируют дифтонги?,оральные и назальные
5,4857,71431,По мнению Швейцарского банковского объединения...,На что в дальнейшем смогут рассчитывать зарубе...,на предсказуемость швейцарского правового порядка
6,6674,58761,"Судебная власть организована в форме пирамиды,...",Что имеет каждый судебный округ?,суд первой инстанции
7,6980,18725,Сверхкороткие импульсы лазерного излучения исп...,"Что используются и в военных целях, например, ...",Лазеры
8,1154,28808,Разработчики и контролёры‐испытатели проприета...,"Что в гораздо большей степени зависит от того,...",эффективность разработки
9,13109,42131,"Лингвистика языка изучает язык как код, то ест...",Что в лингвистике речи понимается под речевыми...,Статический аспект речи — предмет лингвистики ...


In [ ]:
data.info()

In [85]:
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
stemmer = nltk.stem.porter.PorterStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

In [86]:
from nltk.tokenize import sent_tokenize

In [91]:
def normalize_answer(text):
    return ' '.join(re.findall(r"\w+", text)).lower()

def normalize_answer_no_lower(text):
    return ' '.join(re.findall(r"\w+", text))

def sentence_to_word(sentences):
    sentences_in_words = list()
    for sentence in sentences:
        sentences_in_words.append(normalize_answer(sentence).split())
    return sentences_in_words

def safe_split(regex, text):
    res = []
    sear = regex.search(text)
    while sear:
        res.append(text[:sear.end()])
        text = text[sear.end():]
        sear = regex.search(text)
    res.append(text)
    return res


re1 = re.compile("""
    (?:
        (?:
            (?<!\\d(?:р|г|к))
            (?<!и\\.т\\.(?:д|п))
            (?<!и(?=\\.т\\.(?:д|п)\\.))
            (?<!и\\.т(?=\\.(?:д|п)\\.))
            (?<!руб|коп)
        \\.) |
        [!?\\n]
    )+
    """, re.X)


def text_to_sentence(text):
    # sentences = [a for a in re.split(r'\. ', text) if a]
    sentences = safe_split(re1, text)
#     mytext = "Bonjour M. Adam, comment allez-vous? J'espère que tout va bien. Aujourd'hui est un bon jour."
#     sentences = sent_tokenize(text,"russian")
    return [s.strip() for s in sentences if s.strip() != '']

def get_max_match_sentance(data_row):
    sentences = text_to_sentence(data_row["paragraph"])
    sentences_in_words = sentence_to_word(sentences)
    question_in_words = sentence_to_word([data_row["question"]])[0]

    max_overlap = None
    max_match_sentance_id = None

    question_words = set(question_in_words)
    for sentance_id in range(len(sentences_in_words)):
        sentence_words = set(sentences_in_words[sentance_id])
        overlap = len(sentence_words.intersection(question_words))
        if max_overlap is None or overlap > max_overlap:
            max_overlap = overlap
            max_match_sentance_id = sentance_id

    prediction_sentence = sentences[max_match_sentance_id]
    prediction_words = ' '.join([x for x in normalize_answer_no_lower(prediction_sentence).split() if x not in normalize_answer_no_lower(data_row["question"]).split()])

    if prediction_words == '':
        return prediction_sentence, sentences[max_match_sentance_id]
    else:
        return prediction_sentence, prediction_words

In [92]:
data['prediction_sentence'] = ''
data['prediction_words'] = ''
# calculate alone nlpS - by sentences standalone
for index, row in tqdm.tqdm(data.iterrows(), total=data.shape[0], desc="calculating alone nlps"):
    data.loc[index, 'prediction_sentence'], data.loc[index, 'prediction_words'] = get_max_match_sentance(row)

calculating alone nlps:   0%|          | 0/50364 [00:00<?, ?it/s]


LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/PY3/russian.pickle' not found.
  Please use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/home/local/X5/nikita.melentev/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************

In [50]:
# by words = 0.0072864980810449648 - max
f1_score(data.answer, data.prediction_words, average='macro')

/opt/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.0072864980810449648

In [7]:
data.paragraph.values[0]

'Первые упоминания о строении человеческого тела встречаются в Древнем Египте. В XXVII веке до н. э. египетский врач Имхотеп описал некоторые органы и их функции, в частности головной мозг, деятельность сердца, распространение крови по сосудам. В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека. В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге.'

In [8]:
data.question.values[0]

'Где встречаются первые упоминания о строении человеческого тела?'

In [9]:
' '.join([x for x in data.prediction_sentence.values[0].split(' ') if x not in data.question[0].split(' ')])

'Первые тела в Древнем Египте.'

In [51]:
data[data.prediction_words != '']

,paragraph_id,question_id,paragraph,question,answer,prediction_sentence,prediction_words
0,14754,60544,Первые упоминания о строении человеческого тел...,Где встречаются первые упоминания о строении ч...,в Древнем Египте,Первые упоминания о строении человеческого тел...,Первые в Древнем Египте
1,13859,1604,Телескоп имеет модульную структуру и содержит ...,Как называется корректирующая оптическая систе...,COSTAR,Один из отсеков в течение долгого времени (199...,Один из отсеков течение долгого времени 2009 г...
2,8310,30350,Критики теории Вегенера поставили во главу угл...,Какая теория была отвергнута после смерти Веге...,теория дрейфа материков,После смерти Альфреда Вегенера теория дрейфа м...,После Альфреда дрейфа материков получив статус...
3,8766,53270,При нагревании кусочки янтаря становятся очень...,Чему не уступают по красоте изделия из прессов...,изделиям из монолитных камней,Изделия из прессованного янтаря не уступают по...,Изделия изделиям монолитных камней однако буду...
4,14719,75698,Нисходящие дифтонги со слабым гласным /i/ и /u...,На какие группы классифицируют дифтонги?,оральные и назальные,"Как и монофтонги, дифтонги делятся на две груп...",Как и монофтонги делятся на две оральные и наз...
5,4857,71431,По мнению Швейцарского банковского объединения...,На что в дальнейшем смогут рассчитывать зарубе...,на предсказуемость швейцарского правового порядка,"Очень важно, что соглашение находится в рамках...",Очень важно соглашение находится рамках швейца...
6,6674,58761,"Судебная власть организована в форме пирамиды,...",Что имеет каждый судебный округ?,суд первой инстанции,Каждый судебный округ (фр.,Каждый фр
7,6980,18725,Сверхкороткие импульсы лазерного излучения исп...,"Что используются и в военных целях, например, ...",Лазеры,"Лазеры используются и в военных целях, наприме...",Лазеры
8,1154,28808,Разработчики и контролёры‐испытатели проприета...,"Что в гораздо большей степени зависит от того,...",эффективность разработки,Для открытого же проекта круг и взаимное распо...,Для открытого же проекта круг и взаимное распо...
9,13109,42131,"Лингвистика языка изучает язык как код, то ест...",Что в лингвистике речи понимается под речевыми...,Статический аспект речи — предмет лингвистики ...,"Лингвистика речи изучает речевую деятельность,...",Лингвистика изучает речевую деятельность то ес...


## NN

In [6]:
import functools
from collections import Counter

In [7]:
@functools.lru_cache(maxsize=2 ** 19)
def uniq_words(text):
    return set(re.findall("\w+", text))

def calculate_idfs(data):
    counter = Counter()
    uniq_paragraphs = data['paragraph'].unique()
    uniq_questions = data['question'].unique()
    
    for paragraph in tqdm.tqdm(uniq_paragraphs, desc="calc idf for paragraph"):
        set_words = uniq_words(paragraph)
        counter.update(set_words)
    
    for question in tqdm.tqdm(uniq_questions, desc="calc idf for question"):
        set_words = uniq_words(question)
        counter.update(set_words)
        
    num_docs = uniq_paragraphs.shape[0] + uniq_questions.shape[0]
    idfs = {}
    for word in counter:
        idfs[word] = np.log(num_docs / counter[word])
        
    return idfs

In [8]:
idfs = calculate_idfs(data)

calc idf for question: 100%|██████████| 50361/50361 [00:01<00:00, 46374.02it/s]


In [9]:
data['answer_idfs'] = data['answer'].map(lambda x: [idfs.get(word, 0.0) for word in x.split(' ')])
data['prediction_words_idfs'] = data['prediction_words'].map(lambda x: [idfs.get(word, 0.0) for word in x.split(' ')])
data['prediction_sentence_idfs'] = data['prediction_sentence'].map(lambda x: [idfs.get(word, 0.0) for word in x.split(' ')])

In [10]:
data[['answer', 'answer_idfs', 'prediction_words', 'prediction_words_idfs', 'prediction_sentence', 'prediction_sentence_idfs']]

,answer,answer_idfs,prediction_words,prediction_words_idfs,prediction_sentence,prediction_sentence_idfs
0,в Древнем Египте,"[0.984813243671, 7.62541002558, 7.62541002558]",Первые в Древнем Египте,"[6.52679773691, 0.984813243671, 7.62541002558,...",Первые упоминания о строении человеческого тел...,"[6.52679773691, 7.69686898956, 3.24123053754, ..."
1,COSTAR,[9.60641149444],Один из отсеков течение долгого времени 2009 г...,"[6.75859935097, 2.43360326107, 9.04679570651, ...",Один из отсеков в течение долгого времени (199...,"[6.75859935097, 2.43360326107, 9.04679570651, ..."
2,теория дрейфа материков,"[5.51624230363, 8.35364852595, 8.69012076257]",После Альфреда дрейфа материков получив статус...,"[4.63486358905, 9.20094638633, 8.35364852595, ...",После смерти Альфреда Вегенера теория дрейфа м...,"[4.63486358905, 5.36868834938, 9.20094638633, ..."
3,изделиям из монолитных камней,"[9.89409356689, 2.43360326107, 10.9927058556, ...",Изделия изделиям монолитных камней однако буду...,"[9.89409356689, 9.89409356689, 10.9927058556, ...",Изделия из прессованного янтаря не уступают по...,"[9.89409356689, 2.43360326107, 9.89409356689, ..."
4,оральные и назальные,"[10.9927058556, 1.42092192757, 10.299558675]",Как и монофтонги делятся на две оральные и наз...,"[2.7637282122, 1.42092192757, 10.9927058556, 6...","Как и монофтонги, дифтонги делятся на две груп...","[2.7637282122, 1.42092192757, 0.0, 8.594810582..."
5,на предсказуемость швейцарского правового порядка,"[1.74199158945, 10.299558675, 8.35364852595, 8...",Очень важно соглашение находится рамках швейца...,"[7.94818341784, 7.5915084739, 7.18604336579, 4...","Очень важно, что соглашение находится в рамках...","[7.94818341784, 0.0, 2.57643858274, 7.18604336..."
6,суд первой инстанции,"[6.71603973655, 5.11777512471, 7.9016634022]",Каждый фр,"[6.91516841166, 6.5859866083]",Каждый судебный округ (фр.,"[6.91516841166, 8.35364852595, 7.77383003069, ..."
7,Лазеры,[9.60641149444],Лазеры,[9.60641149444],"Лазеры используются и в военных целях, наприме...","[9.60641149444, 5.37230498985, 1.42092192757, ..."
8,эффективность разработки,"[6.44941107329, 6.46010636241]",Для открытого же проекта круг и взаимное распо...,"[3.9228317271, 7.40918691711, 3.73640861587, 6...",Для открытого же проекта круг и взаимное распо...,"[3.9228317271, 7.40918691711, 3.73640861587, 6..."
9,Статический аспект речи — предмет лингвистики ...,"[0.0, 8.28465565446, 6.23053192076, 0.0, 6.967...",Лингвистика изучает речевую деятельность то ес...,"[10.299558675, 6.67521774203, 10.299558675, 5....","Лингвистика речи изучает речевую деятельность,...","[10.299558675, 6.23053192076, 6.67521774203, 1..."


In [11]:
dataX = []
dataY = []

seq_length = 3

for row in tqdm.tqdm(data[:20000].iterrows(), total = 20000):
    
    prediction_sentence = row[1]['prediction_sentence'].split(' ')
    answer = row[1]['answer']

    for i in range(0, len(prediction_sentence) - seq_length, 1):
        seq_in = prediction_sentence[i:i + seq_length]
        seq_out = prediction_sentence[i + seq_length] in answer
        dataX.append([idfs.get(word, 0.0) for word in seq_in])
        dataY.append(int(seq_out))
        
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

100%|██████████| 20000/20000 [00:04<00:00, 4791.82it/s]

Total Patterns:  396504


In [14]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
# X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [12]:
import numpy

In [13]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [15]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# # define the checkpoint
# filepath="../results/b_weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

# # fit the model
# model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
396504/396504 [==============================] - 171s - loss: 0.4538   
Epoch 2/50
396504/396504 [==============================] - 171s - loss: 0.4533   
Epoch 3/50
396504/396504 [==============================] - 170s - loss: 0.4531   
Epoch 4/50
396504/396504 [==============================] - 171s - loss: 0.4529   
Epoch 5/50
396504/396504 [==============================] - 170s - loss: 0.4528   
Epoch 6/50
396504/396504 [==============================] - 171s - loss: 0.4527   
Epoch 7/50
396504/396504 [==============================] - 171s - loss: 0.4527   
Epoch 8/50
396504/396504 [==============================] - 172s - loss: 0.4525   
Epoch 9/50
396504/396504 [==============================] - 173s - loss: 0.4525   
Epoch 10/50
396504/396504 [==============================] - 172s - loss: 0.4525   
Epoch 11/50
396504/396504 [==============================] - 173s - loss: 0.4524   
Epoch 12/50
396504/396504 [==============================] - 171s - loss: 0.4524   
E

396504/396504 [==============================] - 170s - loss: 0.4520   
Epoch 34/50
396504/396504 [==============================] - 171s - loss: 0.4521   
Epoch 35/50
396504/396504 [==============================] - 171s - loss: 0.4521   
Epoch 36/50
242688/396504 [=================>............] - ETA: 66s - loss: 0.4516

In [17]:
# load the network weights
filename = "../results/b_weights-improvement-45-0.4520-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
data

,paragraph_id,question_id,paragraph,question,answer,prediction_sentence,prediction_words,answer_idfs,prediction_words_idfs,prediction_sentence_idfs
0,14754,60544,Первые упоминания о строении человеческого тел...,Где встречаются первые упоминания о строении ч...,в Древнем Египте,Первые упоминания о строении человеческого тел...,Первые в Древнем Египте,"[0.984813243671, 7.62541002558, 7.62541002558]","[6.52679773691, 0.984813243671, 7.62541002558,...","[6.52679773691, 7.69686898956, 3.24123053754, ..."
1,13859,1604,Телескоп имеет модульную структуру и содержит ...,Как называется корректирующая оптическая систе...,COSTAR,Один из отсеков в течение долгого времени (199...,Один из отсеков течение долгого времени 2009 г...,[9.60641149444],"[6.75859935097, 2.43360326107, 9.04679570651, ...","[6.75859935097, 2.43360326107, 9.04679570651, ..."
2,8310,30350,Критики теории Вегенера поставили во главу угл...,Какая теория была отвергнута после смерти Веге...,теория дрейфа материков,После смерти Альфреда Вегенера теория дрейфа м...,После Альфреда дрейфа материков получив статус...,"[5.51624230363, 8.35364852595, 8.69012076257]","[4.63486358905, 9.20094638633, 8.35364852595, ...","[4.63486358905, 5.36868834938, 9.20094638633, ..."
3,8766,53270,При нагревании кусочки янтаря становятся очень...,Чему не уступают по красоте изделия из прессов...,изделиям из монолитных камней,Изделия из прессованного янтаря не уступают по...,Изделия изделиям монолитных камней однако буду...,"[9.89409356689, 2.43360326107, 10.9927058556, ...","[9.89409356689, 9.89409356689, 10.9927058556, ...","[9.89409356689, 2.43360326107, 9.89409356689, ..."
4,14719,75698,Нисходящие дифтонги со слабым гласным /i/ и /u...,На какие группы классифицируют дифтонги?,оральные и назальные,"Как и монофтонги, дифтонги делятся на две груп...",Как и монофтонги делятся на две оральные и наз...,"[10.9927058556, 1.42092192757, 10.299558675]","[2.7637282122, 1.42092192757, 10.9927058556, 6...","[2.7637282122, 1.42092192757, 0.0, 8.594810582..."
5,4857,71431,По мнению Швейцарского банковского объединения...,На что в дальнейшем смогут рассчитывать зарубе...,на предсказуемость швейцарского правового порядка,"Очень важно, что соглашение находится в рамках...",Очень важно соглашение находится рамках швейца...,"[1.74199158945, 10.299558675, 8.35364852595, 8...","[7.94818341784, 7.5915084739, 7.18604336579, 4...","[7.94818341784, 0.0, 2.57643858274, 7.18604336..."
6,6674,58761,"Судебная власть организована в форме пирамиды,...",Что имеет каждый судебный округ?,суд первой инстанции,Каждый судебный округ (фр.,Каждый фр,"[6.71603973655, 5.11777512471, 7.9016634022]","[6.91516841166, 6.5859866083]","[6.91516841166, 8.35364852595, 7.77383003069, ..."
7,6980,18725,Сверхкороткие импульсы лазерного излучения исп...,"Что используются и в военных целях, например, ...",Лазеры,"Лазеры используются и в военных целях, наприме...",Лазеры,[9.60641149444],[9.60641149444],"[9.60641149444, 5.37230498985, 1.42092192757, ..."
8,1154,28808,Разработчики и контролёры‐испытатели проприета...,"Что в гораздо большей степени зависит от того,...",эффективность разработки,Для открытого же проекта круг и взаимное распо...,Для открытого же проекта круг и взаимное распо...,"[6.44941107329, 6.46010636241]","[3.9228317271, 7.40918691711, 3.73640861587, 6...","[3.9228317271, 7.40918691711, 3.73640861587, 6..."
9,13109,42131,"Лингвистика языка изучает язык как код, то ест...",Что в лингвистике речи понимается под речевыми...,Статический аспект речи — предмет лингвистики ...,"Лингвистика речи изучает речевую деятельность,...",Лингвистика изучает речевую деятельность то ес...,"[0.0, 8.28465565446, 6.23053192076, 0.0, 6.967...","[10.299558675, 6.67521774203, 10.299558675, 5....","[10.299558675, 6.23053192076, 6.67521774203, 1..."


In [76]:
for row in tqdm.tqdm(data.iterrows(), total = data.shape[0]):
    
    prediction_sentence = row[1]['prediction_sentence'].split(' ')
    prediction_words = row[1]['prediction_words'].split(' ')
    
    possible_answer_words = {}
    for i in range(0, len(prediction_sentence) - seq_length + 1, 1):
        seq_in = prediction_sentence[i:i + seq_length]
        x = [idfs.get(word, 0.0) for word in seq_in]
        y = model.predict(numpy.reshape(x, (1, seq_length, 1)))[0]
        
        possible_answer_words.update({seq_in[2].strip(',.()-—'):y[1]})
        
#     print(possible_answer_words)
#     print(prediction_words)
#     print([possible_answer_words.get(word) for word in prediction_words])
#     print(row[1]['answer'])
    
    prediction_words_is_answer = [possible_answer_words.get(word) for word in prediction_words]
    prediction_words_mean = np.mean([x for x in prediction_words_is_answer if x is not None])
    
    answer = ' '.join([word for word in prediction_words if possible_answer_words.get(word) is not None and  possible_answer_words.get(word) > prediction_words_mean])
    data.loc[row[0], 'prediction_words_idf'] = answer
    


  0%|          | 6/50364 [00:00<38:37, 21.73it/s]/opt/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 50364/50364 [30:34<00:00, 23.67it/s] 


In [80]:
data

,paragraph_id,question_id,paragraph,question,answer,prediction_sentence,prediction_words,answer_idfs,prediction_words_idfs,prediction_sentence_idfs,prediction_words_idf
0,14754,60544,Первые упоминания о строении человеческого тел...,Где встречаются первые упоминания о строении ч...,в Древнем Египте,Первые упоминания о строении человеческого тел...,Первые в Древнем Египте,"[0.984813243671, 7.62541002558, 7.62541002558]","[6.52679773691, 0.984813243671, 7.62541002558,...","[6.52679773691, 7.69686898956, 3.24123053754, ...",Древнем Египте
1,13859,1604,Телескоп имеет модульную структуру и содержит ...,Как называется корректирующая оптическая систе...,COSTAR,Один из отсеков в течение долгого времени (199...,Один из отсеков течение долгого времени 2009 г...,[9.60641149444],"[6.75859935097, 2.43360326107, 9.04679570651, ...","[6.75859935097, 2.43360326107, 9.04679570651, ...",отсеков занимала COSTAR
2,8310,30350,Критики теории Вегенера поставили во главу угл...,Какая теория была отвергнута после смерти Веге...,теория дрейфа материков,После смерти Альфреда Вегенера теория дрейфа м...,После Альфреда дрейфа материков получив статус...,"[5.51624230363, 8.35364852595, 8.69012076257]","[4.63486358905, 9.20094638633, 8.35364852595, ...","[4.63486358905, 5.36868834938, 9.20094638633, ...",Альфреда дрейфа материков получив маргинальной...
3,8766,53270,При нагревании кусочки янтаря становятся очень...,Чему не уступают по красоте изделия из прессов...,изделиям из монолитных камней,Изделия из прессованного янтаря не уступают по...,Изделия изделиям монолитных камней однако буду...,"[9.89409356689, 2.43360326107, 10.9927058556, ...","[9.89409356689, 9.89409356689, 10.9927058556, ...","[9.89409356689, 2.43360326107, 9.89409356689, ...",изделиям монолитных стоить
4,14719,75698,Нисходящие дифтонги со слабым гласным /i/ и /u...,На какие группы классифицируют дифтонги?,оральные и назальные,"Как и монофтонги, дифтонги делятся на две груп...",Как и монофтонги делятся на две оральные и наз...,"[10.9927058556, 1.42092192757, 10.299558675]","[2.7637282122, 1.42092192757, 10.9927058556, 6...","[2.7637282122, 1.42092192757, 0.0, 8.594810582...",монофтонги делятся оральные назальные
5,4857,71431,По мнению Швейцарского банковского объединения...,На что в дальнейшем смогут рассчитывать зарубе...,на предсказуемость швейцарского правового порядка,"Очень важно, что соглашение находится в рамках...",Очень важно соглашение находится рамках швейца...,"[1.74199158945, 10.299558675, 8.35364852595, 8...","[7.94818341784, 7.5915084739, 7.18604336579, 4...","[7.94818341784, 0.0, 2.57643858274, 7.18604336...",швейцарского права дополнительно укрепляет дел...
6,6674,58761,"Судебная власть организована в форме пирамиды,...",Что имеет каждый судебный округ?,суд первой инстанции,Каждый судебный округ (фр.,Каждый фр,"[6.71603973655, 5.11777512471, 7.9016634022]","[6.91516841166, 6.5859866083]","[6.91516841166, 8.35364852595, 7.77383003069, ...",
7,6980,18725,Сверхкороткие импульсы лазерного излучения исп...,"Что используются и в военных целях, например, ...",Лазеры,"Лазеры используются и в военных целях, наприме...",Лазеры,[9.60641149444],[9.60641149444],"[9.60641149444, 5.37230498985, 1.42092192757, ...",
8,1154,28808,Разработчики и контролёры‐испытатели проприета...,"Что в гораздо большей степени зависит от того,...",эффективность разработки,Для открытого же проекта круг и взаимное распо...,Для открытого же проекта круг и взаимное распо...,"[6.44941107329, 6.46010636241]","[3.9228317271, 7.40918691711, 3.73640861587, 6...","[3.9228317271, 7.40918691711, 3.73640861587, 6...",проекта круг потенциальных разработчиков огран...
9,13109,42131,"Лингвистика языка изучает язык как код, то ест...",Что в лингвистике речи понимается под речевыми...,Статический аспект речи — предмет лингвистики ...,"Лингвистика речи изучает речевую деятельность,...",Лингвистика изучает речевую деятельность то ес...,"[0.0, 8.28465565446, 6.23053192076, 0.0, 6.967...","[10.299558675, 6.67521774203, 10.299558

In [81]:
data.loc[data[data.prediction_words_idf == ''].index, 'prediction_words_idf'] = data[data.prediction_words_idf == ''].prediction_words

In [82]:
# by words = 0.0072864980810449648 - max
f1_score(data.answer, data.prediction_words_idf, average='macro')

/opt/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.0076167984301400435